In [1]:
#get data from google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# keras module for building LSTM 
import tensorflow as tf
from tensorflow.keras.utils import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku

# set seeds for reproducability
from numpy.random import seed

import pandas as pd
import numpy as np
import string, os

# load numpy array from csv file
from numpy import loadtxt

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
tf.random.set_seed(2)

## Load prepared dataset

In [4]:
# load array
X = pd.read_csv('/content/drive/MyDrive/data/X_gen.csv')
y = pd.read_csv('/content/drive/MyDrive/data/y_gen.csv')

# print the array
print(X)
print(y)

      suicide rates among university students england wales gone slightly last decade according official statistics reveal young male students significantly likely kill female students 12 months july 2017 figures office national statistics ons revealed 95 stude
0      jon snow daenerys targaryen cersei lannister s...                                                                                                                                                                                                             
1      bank england must ensure green recovery covid1...                                                                                                                                                                                                             
2      congress voted week allow internet service pro...                                                                                                                                                              

In [5]:
len(X)

14858

In [6]:
len(y)

14858

## Tokenise Data

In [7]:
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(X)
inp_sequences[:10]

[[3, 4],
 [3, 4, 5],
 [3, 4, 5, 6],
 [3, 4, 5, 6, 1],
 [3, 4, 5, 6, 1, 7],
 [3, 4, 5, 6, 1, 7, 8],
 [3, 4, 5, 6, 1, 7, 8, 9],
 [3, 4, 5, 6, 1, 7, 8, 9, 10],
 [3, 4, 5, 6, 1, 7, 8, 9, 10, 11],
 [3, 4, 5, 6, 1, 7, 8, 9, 10, 11, 12]]

In [8]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

In [9]:
input_len = max_sequence_len - 1

In [10]:
print(input_len)

34


In [11]:
print(total_words)

33


## LSTM Text Generative Model

In [37]:
gen_model = Sequential()

In [38]:
# Add Input Embedding Layer
gen_model.add(Embedding(total_words, 10, input_length=input_len))

In [40]:
# Add Hidden Layer 1 - LSTM Layer
gen_model.add(LSTM(100, return_sequences=True))

In [41]:
gen_model.add(Dropout(0.1))

In [42]:
gen_model.add(LSTM(100))

In [43]:
# Add Output Layer
gen_model.add(Dense(total_words, activation='softmax'))

In [44]:
gen_model.compile(loss='categorical_crossentropy', optimizer='adam')

In [45]:
gen_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 34, 10)            330       
                                                                 
 lstm_3 (LSTM)               (None, 34, 100)           44400     
                                                                 
 dropout_2 (Dropout)         (None, 34, 100)           0         
                                                                 
 lstm_4 (LSTM)               (None, 100)               80400     
                                                                 
 dense_1 (Dense)             (None, 33)                3333      
                                                                 
Total params: 128,463
Trainable params: 128,463
Non-trainable params: 0
_________________________________________________________________


In [46]:
gen_model.fit(predictors, label, epochs=100, verbose=5)

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

In [47]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len, padding='pre') 
        classes = model.predict(token_list, verbose=0)
        predicted=np.argmax(classes,axis=1)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [48]:
seed_text = "From Balenciaga problematic advertising campaigns to the Kanye West scandal"""

In [49]:
next_words = 10
generate_text(seed_text, next_words, gen_model, input_len)

'From Balenciaga Problematic Advertising Campaigns To The Kanye West Scandal Students Students Students Students Students Students Students Students England Decade'